In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [5]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312      1.0  0.000011                  NaN           NaN
0.000312      NaN       NaN                  1.0 -3.970604e-06
0.000750      2.0  0.000016                  2.0 -3.501435e-06
0.001052      3.0  0.000026                  3.0  1.764192e-06
0.001476      4.0  0.000038                  4.0  6.246484e-06
0.002070      5.0  0.000052                  5.0  1.145732e-05
0.002904      6.0  0.000069                  6.0  1.749111e-05
0.004074      7.0  0.000089                  7.0  2.493673e-05
0.005714      8.0  0.000112                  8.0  3.403144e-05
0.008015      9.0  0.000139                  9.0  4.464038e-05
0.011243     10.0  0.000171                 10.0  5.685051e-05
0.015771     11.0  0.000207                 11.0  7.089259e-05
0.022122     12.0  0.000249                 12.0  8.708337e-05
0.031031     13.0  0.000296                 13.0  1.056316e-04
0.043528     14.0  0.000344                 14.0  1.242592e-04
0.061057     15.0  0.000386                 15.0  1.403381e-04
0.085645     16.0  0.000424                 16.0  1.554926e-04
0.120136     17.0  0.000466                 17.0  1.716111e-04
0.168516     18.0  0.000511                 18.0  1.885451e-04
0.236378     19.0  0.000559                 19.0  2.062490e-04
0.331549     20.0  0.000611                 20.0  2.248443e-04
0.465100     21.0  0.000661                 21.0  2.438501e-04
0.652400     22.0  0.000621                 22.0  2.315875e-04
0.915100     23.0  0.000478                 23.0  1.785513e-04
1.283650     24.0  0.000341                 24.0  1.256818e-04
1.800600     25.0  0.000236                 25.0  8.531458e-05
2.525700     26.0  0.000158                 26.0  5.517612e-05
3.542800     27.0  0.000101                 27.0  3.295297e-05
4.969550     28.0  0.000064                 28.0  1.868456e-05
6.970850     29.0  0.000043                 29.0  1.070508e-05
9.778100     30.0  0.000030                 30.0  5.474737e-06
13.715850    31.0  0.000019                 31.0  1.303022e-06
19.239350    32.0  0.000012                 32.0 -1.858605e-06
26.987250    33.0  0.000009                 33.0 -2.810196e-06
37.855300    34.0  0.000012                 34.0 -1.518420e-06
53.100050    35.0  0.000015                 35.0  4.588271e-07
73.887500    36.0  0.000019                 36.0  2.790776e-06
97.662500    37.0  0.000022                 37.0  4.798332e-06
121.437500   38.0  0.000022                 38.0  5.549865e-06
145.212500   39.0  0.000029                 39.0  7.444977e-06
168.987500   40.0  0.000036                 40.0  9.358619e-06
192.762500   41.0  0.000043                 41.0  1.139964e-05
216.537500   42.0  0.000046                 42.0  1.263291e-05
240.312500   43.0  0.000048                 43.0  1.357396e-05
264.087500   44.0  0.000051                 44.0  1.438180e-05
287.862500   45.0  0.000055                 45.0  1.676207e-05
311.637500   46.0  0.000082                 46.0  3.557714e-05
335.412500   47.0  0.000160                 47.0  9.060844e-05
359.187500   48.0  0.000293                 48.0  1.878735e-04
382.962500   49.0  0.000609                 49.0  4.176611e-04
406.737500   50.0  0.001231                 50.0  8.886416e-04
430.512500   51.0  0.002219                 51.0  1.668749e-03
454.287500   52.0  0.003648                 52.0  2.860670e-03
478.062500   53.0  0.005546                 53.0  4.538240e-03
501.837500   54.0  0.007899                 54.0  6.730675e-03
525.612500   55.0  0.010643                 55.0  9.325690e-03
549.387500   56.0  0.013670                 56.0  1.228605e-02
573.162500   57.0  0.016861                 57.0  1.548225e-02
596.937500   58.0  0.020079                 58.0  1.874983e-02
620.712500   59.0  0.023207                 59.0  2.193658e-02
644.487500   60.0  0.026145                 6

# Fluxes by Layer

CRD                              CLIRAD            \
                         flug          fldg      fnetg       flug      fldg   
pressure     level                                                            
0.000000e+00 1     -18.787640  0.000000e+00 -18.787640        NaN       NaN   
1.000000e-08 1            NaN           NaN        NaN -18.803282  0.000001   
6.244000e-04 2     -18.787640  4.451939e-09 -18.787640 -18.803282  0.000001   
8.759000e-04 3     -18.787640  6.370098e-09 -18.787640 -18.803282  0.000001   
1.228600e-03 4     -18.787640  9.450519e-09 -18.787640 -18.803282  0.000001   
1.723400e-03 5     -18.787640  1.440286e-08 -18.787640 -18.803282  0.000001   
2.417400e-03 6     -18.787640  2.235540e-08 -18.787640 -18.803282  0.000001   
3.390900e-03 7     -18.787640  3.511338e-08 -18.787640 -18.803282  0.000001   
4.756500e-03 8     -18.787640  5.555993e-08 -18.787640 -18.803282  0.000001   
6.672000e-03 9     -18.787640  8.829336e-08 -18.787640 -18.803282  0.000001   
9.358900e-03 10    -18.787640  1.406452e-07 -18.787640 -18.803282  0.000001   
1.312780e-02 11    -18.787640  2.242893e-07 -18.787640 -18.803282  0.000001   
1.841450e-02 12    -18.787640  3.577999e-07 -18.787639 -18.803282  0.000001   
2.583020e-02 13    -18.787640  5.706941e-07 -18.787639 -18.803282  0.000002   
3.623230e-02 14    -18.787640  9.096483e-07 -18.787639 -18.803282  0.000002   
5.082340e-02 15    -18.787640  1.440999e-06 -18.787638 -18.803282  0.000002   
7.129060e-02 16    -18.787640  2.252620e-06 -18.787637 -18.803282  0.000002   
1.000000e-01 17    -18.787639  3.477500e-06 -18.787636 -18.803282  0.000003   
1.402710e-01 18    -18.787639  5.324854e-06 -18.787634 -18.803282  0.000003   
1.967600e-01 19    -18.787638  8.110076e-06 -18.787630 -18.803282  0.000005   
2.759970e-01 20    -18.787637  1.230775e-05 -18.787625 -18.803282  0.000006   
3.871000e-01 21    -18.787636  1.862926e-05 -18.787617 -18.803281  0.000009   
5.431000e-01 22    -18.787633  2.807851e-05 -18.787605 -18.803281  0.000013   
7.617000e-01 23    -18.787629  4.064750e-05 -18.787589 -18.803280  0.000018   
1.068500e+00 24    -18.787626  5.494944e-05 -18.787571 -18.803279  0.000024   
1.498800e+00 25    -18.787624  7.038117e-05 -18.787554 -18.803280  0.000031   
2.102400e+00 26    -18.787624  8.691990e-05 -18.787537 -18.803281  0.000038   
2.949000e+00 27    -18.787626  1.045842e-04 -18.787521 -18.803283  0.000046   
4.136600e+00 28    -18.787630  1.233863e-04 -18.787507 -18.803287  0.000054   
5.802500e+00 29    -18.787638  1.441208e-04 -18.787494 -18.803293  0.000064   
8.139200e+00 30    -18.787651  1.686027e-04 -18.787482 -18.803302  0.000076   
1.141700e+01 31    -18.787669  1.986091e-04 -18.787471 -18.803315  0.000091   
1.601470e+01 32    -18.787696  2.358834e-04 -18.787460 -18.803333  0.000109   
2.246400e+01 33    -18.787734  2.831573e-04 -18.787451 -18.803359  0.000134   
3.151050e+01 34    -18.787789  3.478606e-04 -18.787441 -18.803397  0.000169   
4.420010e+01 35    -18.787867  4.437352e-04 -18.787423 -18.803451  0.000221   
6.200000e+01 36    -18.787979  5.880651e-04 -18.787391 -18.803532  0.000303   
8.577500e+01 37    -18.788131  7.944337e-04 -18.787337 -18.803649  0.000428   
1.095500e+02 38    -18.788287  1.011463e-03 -18.787276 -18.803773  0.000565   
1.333250e+02 39    -18.788448  1.234831e-03 -18.787213 -18.803903  0.000711   
1.571000e+02 40    -18.788676  1.544982e-03 -18.787131 -18.804089  0.000918   
1.808750e+02 41    -18.788984  1.953287e-03 -18.787031 -18.804340  0.001196   
2.046500e+02 42    -18.789389  2.477507e-03 -18.786911 -18.804673  0.001560   
2.284250e+02 43    -18.789876  3.094841e-03 -18.786781 -18.805079  0.002002   
2.522000e+02 44    -18.790451  3.805939e-03 -18.786645 -18.805564  0.002526   
2.759750e+02 45    -18.791130  4.628218e-03 -18.786502 -18.806143  0.003145   
2.997500e+02 46    -18.791934  5.586511e-03 -18.786347 -18.806831  0.003880   
3.235250e+02 47    -18.792882  6.766246e-03 -18.786116 -18.807648  0.004797 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')